In [1]:
import numpy as np
import scipy as sy
import matplotlib.pyplot as plt
import time
import sys
import os
import ast
import json
import copy 
import itertools
from tqdm import *
from collections import defaultdict
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from ipynb.fs.full.Random_Sample_Mapper import *

In [2]:
notebook_path = os.path.abspath("BPR_OPT_Binary_Model.ipynb")
users_items_file_path = os.path.join(os.path.dirname(notebook_path), "data/australian_users_items.json")
items_file_path = os.path.join(os.path.dirname(notebook_path), "data/items_meta_data.json")
users_meta_data_file_path = os.path.join(os.path.dirname(notebook_path), "data/users_meta_data.json")

In [3]:
users_items = []
with open(users_items_file_path, 'r') as data:
    for line in data:
        users_items.append(ast.literal_eval(line))

In [4]:
with open(items_file_path, 'r') as data:
    games_dict = json.load(data)

In [5]:
with open(users_meta_data_file_path, 'r') as file:
    users_meta_data = json.load(file)

In [6]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
playtimesPerItem = defaultdict(dict)
playtimesPerUser = defaultdict(dict)
itemNames = defaultdict(str)

In [7]:
train_usersPerItem = defaultdict(set)
train_itemsPerUser = defaultdict(set)
test_usersPerItem = defaultdict(set)
test_itemsPerUser = defaultdict(set)

In [8]:
for game in games_dict:
    if 'owners' in games_dict[game]:
        usersPerItem[game] = set(games_dict[game]['owners'].keys())

In [9]:
for user in users_items:
    u_id = user['user_id']
    items = [item['item_id'] for item in user['items']]
    itemsPerUser[u_id] = items
    
    items_train, items_test = train_test_split(items, test_size = 0.20)
    train_itemsPerUser[u_id] = items_train
    test_itemsPerUser[u_id] = items_test

    playtimesPerUser[user['user_id']] = dict((item['item_id'], item['playtime_forever']) for item in user['items'])
    for item in user['items']:
        itemNames[item['item_id']] = item['item_name']
        playtimesPerItem[item['item_id']][user['user_id']] = item['playtime_forever']

In [10]:
for user in train_itemsPerUser:
    for item in train_itemsPerUser[user]:
        train_usersPerItem[item].add(user)
        
    for item in test_itemsPerUser[user]:
        test_usersPerItem[item].add(user)

In [11]:
nUsers = len(itemsPerUser)
nItems = len(usersPerItem)
users = list(itemsPerUser.keys())
items = list(usersPerItem.keys())

In [12]:
train_user_item_counts = dict((k, len(v)) for k, v in train_itemsPerUser.items())

In [13]:
test_user_item_counts = dict((k, len(v)) for k, v in test_itemsPerUser.items())

In [14]:
train_datafile = 'data/train_sample_in.tsv'
train_mapout1 = 'data/train_sample_map1.tsv'
train_mapout2 = 'data/train_sample_map2.tsv'
train_outfile = 'data/train_sample_out.tsv'

f = open(train_datafile,'w')
for u, its in train_itemsPerUser.items():
    for i in its:
        print(default_formatter(u,i), file=f)
f.close()

In [15]:
# run two stages of mapreduce
train_mapper = Mapper(train_user_item_counts)
mapreduce(train_datafile, train_mapout1, mapper=train_mapper, reducer=reducer)
mapreduce(train_datafile, train_mapout2, mapper=indicator_mapper)  # map the data again
mapreduce([train_mapout1, train_mapout2], train_outfile, reducer=indicator_reducer)

In [16]:
test_sample = []

for u, its in test_itemsPerUser.items():
    for i in its:
        j = random.choice(items)
        test_sample.append((u, i, j))

In [17]:
class ExternalSchedule(object):

    def __init__(self, filepath):
        self.filepath = filepath
        
    def generate_random_samples(self, size=None, rand=True):
        f = open(self.filepath)
        samples = [map(str, line.strip().split()) for line in f]
        if rand:
            random.shuffle(samples)  # important!
        if size is None:
            size = len(samples)
        for u, i, j in samples[:size]:
            yield u, i, j

In [18]:
sampler = ExternalSchedule(train_outfile)  # schedule is one-indexed

In [19]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [20]:
def trim(u, i, j):
    return u[1:len(u)-1], i[2:len(i)-2], j[1:len(j)-2]

In [59]:
def binary_prediction(u, i, j):
    probability = sigmoid(prediction(u, i, j))
    if probability > 0.49:
        return 0
    else:
        return 1

In [60]:
def binary_label(u, i, j):
    c = Counter(itemsPerUser[u])
    return c[j]

In [61]:
def accuracy(sample):
    predictions = []
    labels = []
    marks = []
    for u, i, j in sample:
        probability = sigmoid(prediction(u, i, j))
        marks.append(probability)
        predict = binary_prediction(u, i, j)
        label = binary_label(u, i, j)
        predictions.append(predict)
        labels.append(label)
        
    differences = [1 if x == y else 0 for x, y in zip(labels, predictions)]
    return sum(differences) / len(differences)

In [43]:
def auc(type="complete"):
    auc = 0.0
    for user in tqdm(itemsPerUser):
        y_pred = np.zeroes(nItems)
        if type == "complete":
            user_latent = np.array(userGamma[user])
            items_latent = np.array([x for x in list(itemGamma.values())])
            items_biases = np.array(list(itemBiases.values()))
            y_pred = user_latent.dot(items_latent.T) + items_biases
        else:
            y_pred = inner
        
        c = Counter(itemsPerUser[user])
        y_true = np.array([c[i] for i in items])
        
        if len(np.unique(y_true)) == 1: # bug in roc_auc_score
            auc += accuracy_score(y_true, np.rint(y_pred))
        else:
            auc += roc_auc_score(y_true, y_pred)

    auc /= nUsers
    return auc

## Sigmoid Function

\begin{equation*}
\sigma(x) = \frac{1}{1 + e^{-x}}
\end{equation*}

In [25]:
def sigmoid(x):
    #Numerically stable sigmoid function.
    #Taken from: https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    if x >= 0:
        z = np.exp(-x)
        return 1 / (1 + z)
    else:
        # if x is less than zero then z will be small, denom can't be
        # zero because it's 1+z.
        z = np.exp(x)
        return z / (1 + z)

# Simple (Biase Only) Latent Factor Model with Binary Classification

In [26]:
itemBiases = defaultdict(float)

In [27]:
def unpack(theta):
    global itemBiases
    itemBiases = dict(zip(items, theta))

## Prediction Function

\begin{equation*}
f(i, j) = \beta_i - \beta_j
\end{equation*}

\begin{equation*}
p(i >_u j) = \sigma(f(i, j))
\end{equation*}

In [28]:
def prediction(u, item_i, item_j):
    return itemBiases[item_i] - itemBiases[item_j]

\begin{equation*}
\text{BPR-OPT} := \text{argmax} \ln(\sigma(\beta_i - \beta_j))
\end{equation*}

\begin{equation*}
\text{Cost Function}:= \sum_{u,i,j} ln\left( \frac{1}{1 + e^{\beta_j - \beta_i}} \right)
\end{equation*}

## Cost Function

In [29]:
def cost(theta):
    unpack(theta)
    cost = 0
    for u, i, j in sampler.generate_random_samples():
        u, i, j = trim(u, i ,j)
        x = prediction(u, i, j)

        cost += np.log(sigmoid(x))
        
    print(-cost)
    
    return -cost

\begin{equation*}
\frac{\partial }{\partial x} ln\sigma(x) = \frac{1}{1 + e^x} = \sigma(-x)
\end{equation*}

## Partial Derivatives

\begin{equation*}
\frac{\partial f}{\partial \beta_i} = \frac{e^{\beta_j - \beta_i}}{1 + e^{\beta_j - \beta_i}} = \frac{1}{1 + e^{\beta_i - \beta_j}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_j} = -\frac{e^{\beta_j - \beta_i}}{1 + e^{\beta_j - \beta_i}} = -\frac{1}{1 + e^{\beta_i - \beta_j}}
\end{equation*}

In [30]:
def derivative(theta):
    unpack(theta)
    dItemBiases = defaultdict(float)
    for u, i, j in sampler.generate_random_samples():
        u, i, j = trim(u, i ,j)
        x = prediction(u, i, j)
        dbase = sigmoid(-x)
        dItemBiases[i] += dbase
        dItemBiases[j] += -dbase
    dtheta = [dItemBiases[i] for i in items]
    return np.array(dtheta)

In [31]:
res, f, d = sy.optimize.fmin_l_bfgs_b(cost, [0.0]*nItems, derivative)

923790.5186171636
935004.4360156591
924560.5806579089
923859.4476826115
923796.8287255677
923791.0974415451
923790.5717072844
923790.523470556
923790.5190453929
923790.5186477633
923790.518616957
923790.5186190616
923790.518618296
923790.5186169553


In [32]:
d

{'funcalls': 14,
 'grad': array([-10.5       ,  48.49999999, -13.        , ...,  -0.5       ,
         -0.5       ,   0.        ]),
 'nit': 1,
 'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
 'warnflag': 0}

In [33]:
unpack(res)

In [34]:
sample = []
for u, i, j in sampler.generate_random_samples():
    u, i, j = trim(u, i ,j)
    sample.append((u, i, j))

In [62]:
accuracy(sample)

0.9464497414364906

In [63]:
accuracy(test_sample)

0.9718797530216702

# Complete Latent Factor Model with Binary Classification

A simple non-biased latent factor model that is wrapped into a binary function (sigmoid function) as a base line model, using popularity as the item's sole feature

In [64]:
itemBiases = defaultdict(float)

For each user and item we now have a low dimensional descriptor (representing that user's preferences, and that item's properties), of dimension K.

In [65]:
userGamma = {}
itemGamma = {}

In [66]:
K = 2

In [67]:
for u in itemsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [68]:
for i in usersPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

We'll use another library in this example to perform gradient descent. This library requires that we pass it a "flat" parameter vector (theta) containing all of our parameters. This utility function just converts between a flat feature vector, and our model parameters, i.e., it "unpacks" theta into our offset and bias parameters.

In [69]:
def unpack(theta):
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    itemBiases = dict(zip(items, theta[0:index + nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index + K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index + K]
        index += K

## Prediction Function

\begin{equation*}
f(u, i, j) = \gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)
\end{equation*}

\begin{equation*}
p(i >_u j) = \sigma(f(u, i, j))
\end{equation*}

In [81]:
def prediction(user, item_i, item_j):
    return inner(userGamma[user], itemGamma[item_i]) + itemBiases[item_i] - (inner(userGamma[user], itemGamma[item_j]) + itemBiases[item_j]) 

## Cost Function

\begin{equation*}
\text{BPR-OPT} := \text{argmax} \ln(\sigma(\gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)))
\end{equation*}

\begin{equation*}
\text{Cost Function}:= \sum_{u,i,j} ln\left( \frac{1}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}} \right)
\end{equation*}

In [71]:
def cost(theta):
    unpack(theta)
    cost = 0
    for u, i, j in sampler.generate_random_samples(rand=False):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i, j)

        cost += np.log(sigmoid(x))
        
    print(-cost)
        
    return -cost

## Partial Derivatives

\begin{equation*}
\frac{\partial f}{\partial \gamma_{u,k}} = \frac{(\gamma_{i,k} - \gamma_{j,k}) \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{i,k}} = \frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{j,k}} = -\frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_i} = \frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_j} = -\frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

In [72]:
def derivative(theta):
    unpack(theta)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in users:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in items:
        dItemGamma[i] = [0.0 for k in range(K)]
    for u, i, j in sampler.generate_random_samples(rand=False):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i ,j)
        dbase = sigmoid(-x)
        dItemBiases[i] += dbase
        dItemBiases[j] += -dbase
        for k in range(K):
            dUserGamma[u][k] += (itemGamma[i][k] - itemGamma[j][k]) * dbase
            dItemGamma_k = userGamma[u][k] * dbase
            dItemGamma[i][k] += dItemGamma_k
            dItemGamma[j][k] += -dItemGamma_k
    dtheta = [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return np.array(dtheta)

In [73]:
complete_res, f, d = sy.optimize.fmin_l_bfgs_b(cost, [0.0]*nItems + # Initialize beta
                                [random.random() * 0.1 - 0.05 for k in range(K*(nUsers + nItems))], # Gamma
                             derivative)

923791.0516614979
935005.0444739005
924561.1261541311
923859.9819097128
923797.3618960822
923791.6305167308
923791.1047714088
923791.0565343753
923791.052108588
923791.0517025344
923791.0516653144
923791.0516618515
923791.0516615309
923791.0516615018
923791.0516614981
923791.0516614979
923791.0516614979
923791.0516614981
923791.0516614979
923791.0516614979
923791.0516614979


In [74]:
d

{'funcalls': 21,
 'grad': array([-1.04980813e+01,  4.84736097e+01, -1.30010494e+01, ...,
         2.38911063e-02,  0.00000000e+00,  0.00000000e+00]),
 'nit': 0,
 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH',
 'warnflag': 2}

In [75]:
unpack(complete_res)

In [82]:
accuracy(test_sample)

0.9718797530216702

In [78]:
accuracy(sample)

0.9464497414364906